# DAX: Ключевые функции Power BI

**CALCULATE + контекст = сила DAX**

## Содержание
1. Фильтры и контекст  
2. Агрегация и расчёты  
3. Временные расчёты (YoY)  
4. Условные расчёты  
5. Self-Join и сложные связи  
6. Расширенные функции  
7. Мера vs Вычисляемый столбец  
8. Иерархия дат

---

## 1. Фильтры и контекст

```dax
Продажи Москва = 
CALCULATE(
    SUM(Sales[Amount]),
    Customers[City] = "Москва"
)
```

> **CALCULATE** — меняет контекст фильтрации  
> **ALL()** — снимает фильтр: `CALCULATE(SUM(...), ALL(Date))`

## 2. Агрегация и расчёты

```dax
% от общего = 
DIVIDE(
    [Продажи],
    CALCULATE([Продажи], ALL(Sales))
)
```

> **DIVIDE** — безопасное деление  
> **ALL(Sales)** — игнорирует фильтры по строкам Sales

## 3. Временные расчёты (YoY)

```dax
-- Базовая мера
Продажи = SUM(Sales[Amount])

Продажи прошлый год = 
CALCULATE(
    [Продажи],
    SAMEPERIODLASTYEAR('Calendar'[Date])
)

YoY % = 
DIVIDE(
    [Продажи] - [Продажи прошлый год],
    [Продажи прошлый год]
)
```

> **SAMEPERIODLASTYEAR**, **DATESYTD**, **TOTALYTD** — временная интеллект  
> **ALL()** — не ставим, если нужна интерактивность (фильтры по регионам, товарам)

## 4. Условные расчёты

```dax
Категория = 
SWITCH(
    TRUE(),
    [Продажи] > 10000, "High",
    [Продажи] > 1000,  "Medium",
    "Low"
)
```

> **SWITCH(TRUE(), ...)** — как CASE WHEN

## 5. Self-Join и сложные связи

```dax
Имя руководителя = 
LOOKUPVALUE(
    Employees[Name],
    Employees[ID], Employees[ManagerID]
)
```

> **LOOKUPVALUE** — как self-join в SQL

## 6. Расширенные функции

- **Агрегация по связанным таблицам**:
  ```dax
  TotalByCategory = SUMX(RELATEDTABLE(Category), Category[Value])
  ```

- **Ранжирование**:
  ```dax
  Rank = RANKX(ALL(Table), [Продажи], , DESC)
  ```

- **Дата-функции**:
  ```dax
  SalesYTD = TOTALYTD([Продажи], 'Calendar'[Date])
  ```

- **IF для условий**:
  ```dax
  Status = IF([Продажи] > 1000, "High", "Low")
  ```

## 7. Мера vs Вычисляемый столбец

| Мера | Вычисляемый столбец |
|------|---------------------|
| Динамична (реагирует на фильтры) | Статична (считается при загрузке) |
| Не занимает память | Занимает место в модели |
| Используй: YoY, % от общего, фильтры | Используй: категории, флаги, ID |

> **Правило:** Если зависит от **контекста** — **мера**. Если **фиксированное значение** — **столбец**.

## 8. Иерархия дат

```dax
-- Таблица дат с русскими месяцами
Calendar = 
VAR MinDate = MIN(Sales[OrderDate])
VAR MaxDate = MAX(Sales[OrderDate])
RETURN
ADDCOLUMNS(
    CALENDAR(MinDate, MaxDate),
    "Year", YEAR([Date]),
    "Quarter", "Q" & FORMAT([Date], "q"),
    "Month", 
        SWITCH(
            MONTH([Date]),
            1, "Январь", 2, "Февраль", 3, "Март", 4, "Апрель",
            5, "Май", 6, "Июнь", 7, "Июль", 8, "Август",
            9, "Сентябрь", 10, "Октябрь", 11, "Ноябрь", 12, "Декабрь"
        ),
    "MonthNum", MONTH([Date]),
    "Day", DAY([Date])
)
```

**Настройка в Power BI:**
1. Связь: `Calendar[Date]` → `Sales[OrderDate]` (one-to-many)  
2. **Modeling → Mark as Date Table** → `Date`  
3. Создать иерархию: `Year → Quarter → Month → Day`  
4. `Month` → **Sort By Column** → `MonthNum`  
5. **File → Options → Data Load → Disable Auto Date/Time**

```dax
-- Меры с иерархией
Revenue = SUM(Sales[Amount])
RevenuePY = CALCULATE([Revenue], SAMEPERIODLASTYEAR('Calendar'[Date]))
Revenue YoY = DIVIDE([Revenue] - [RevenuePY], [RevenuePY])
```

> **Зачем:** Drill-down (год → месяц), русские месяцы, временные расчёты.

---
**Источники:** DAX Guide, Power BI Community.  
*Добавляйте свои меры через PR.*